<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_splitting_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
!pip install langchain==0.3.20
!pip install langchain-community==0.3.20
!pip install pypdf==5.4.0
!pip install beautifulsoup4==4.13.3
!pip install tiktoken==0.9.0

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
import bs4
import warnings

warnings.filterwarnings('ignore')

<br>

# Cargar y dividir documentos con LangChain

Un paso fundamental en un sistema RAG es la carga y división de documentos en fragmentos informativos, los cuales serán posteriormente vectorizados y almacenados en una base de datos.

Seguiremos los siguientes pasos:

**Cargar**: Cargar documentos para su división. Esto se hace con Document Loaders (cargadores de documentos).

**Dividir**: Utizaremos Text Splitters (divisores de texto), los cuales dividen documentos grandes en fragmentos más pequeños. Esto es útil tanto para indexar los datos como para pasarlos a un modelo, ya que los fragmentos grandes son más difíciles de buscar y no caben en la ventana de contexto limitada de un modelo.

## Carga de información de PDF

In [ ]:
# Se carga el documento PDF.
loader = PyPDFLoader('/content/Aviso_de_privacidad.pdf')
pdf_pages = []
async for page in loader.alazy_load():
    pdf_pages.append(page)

In [ ]:
# Se imprime la lista.
pdf_pages

In [ ]:
# Se imprimen los metadatos de una página.
print(f"{pdf_pages[0].metadata}\n")

In [ ]:
# Se imprime el contenido de una página.
print(pdf_pages[0].page_content)

<br>

## Carga de información de página web

In [ ]:
# Se carga la información de una página web.

page_url = "https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

In [ ]:
# Se imprime la información recabada.
docs

Para conocer más opciones de carga de información, puede consultarse:
https://python.langchain.com/docs/concepts/text_splitters/



<br>

# Dividir documentos

Después de haber cargado la información se debe realizar una división de estos en fragmentos informativos más pequeños.

**¿Por qué dividir documentos?**
Existen varias razones para dividir documentos:

* **Manejo de longitudes no uniformes**: Las colecciones de documentos del mundo real suelen contener textos de diferentes tamaños. Dividirlos garantiza un procesamiento consistente entre todos los documentos.

* **Superar las limitaciones del modelo**: Muchos modelos de embeddings y modelos de lenguaje tienen restricciones en el tamaño máximo de entrada. Dividir permite procesar documentos que, de otro modo, excederían esos límites.

* **Mejorar la calidad de la representación**: En documentos largos, la calidad de los embeddings u otras representaciones puede degradarse al intentar capturar demasiada información. Dividirlos puede generar representaciones más precisas y enfocadas de cada sección.

* **Aumentar la precisión en la recuperación de información**: En sistemas de recuperación de información, dividir documentos puede mejorar la granularidad de los resultados, permitiendo una correspondencia más precisa entre consultas y secciones relevantes del documento.

* **Optimizar los recursos computacionales**: Trabajar con fragmentos más pequeños de texto puede ser más eficiente en memoria y permite una mejor paralelización de las tareas de procesamiento.

# Extracción basada en longitud

La estrategia más intuitiva es dividir los documentos en función de su longitud. Este enfoque, simple pero eficaz, garantiza que cada fragmento no supere un límite de tamaño especificado. Los principales beneficios de la división basada en longitud son:

* Implementación sencilla

* Tamaños de fragmento consistentes

* Fácil adaptación a los requisitos de diferentes modelos

Tipos de división basada en longitud:
* Basada en tokens: Divide el texto según el número de tokens, lo cual es útil al trabajar con modelos de lenguaje.

* Basada en caracteres: Divide el texto según el número de caracteres, lo cual puede ser más consistente entre distintos tipos de texto.

## División basada en tokens

In [ ]:
# Se divide el texto utilizando el tokenizador tiktoken.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=1000, chunk_overlap=200
)
texts = text_splitter.split_documents(pdf_pages)

In [ ]:
texts[0].page_content

In [ ]:
print(f"Se ha dividido la información en {len(texts)} documentos (chunks).")

# División basada en caracteres y estructura del texto

El texto está naturalmente organizado en unidades jerárquicas como párrafos, oraciones y palabras. Podemos aprovechar esta estructura inherente para definir una estrategia de división que mantenga el flujo natural del lenguaje, preserve la coherencia semántica dentro de cada fragmento y se adapte a distintos niveles de granularidad del texto. El RecursiveCharacterTextSplitter de LangChain implementa este concepto:

* RecursiveCharacterTextSplitter intenta mantener intactas las unidades más grandes (por ejemplo, los párrafos).

* Si una unidad supera el tamaño máximo del fragmento, pasa al siguiente nivel (por ejemplo, las oraciones).

* Este proceso continúa hasta el nivel de las palabras, si es necesario.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(pdf_pages)

In [ ]:
texts[0].page_content

In [ ]:
print(f"Se ha dividido la información en {len(texts)} sub-documentos.")

### Diferencia entre dividir por tokens y caracteres

🔤 División por caracteres:
* Se basa en la cantidad de letras, espacios y símbolos que contiene el texto.
* Es una forma muy directa y uniforme de dividir.
* No considera palabras completas, estructuras lingüísticas ni unidades semánticas.

Ejemplo: "Hola, ¿cómo estás?" tiene 18 caracteres (incluyendo signos y espacios).

🧱 División por tokens:
* Se basa en el número de tokens generados por un tokenizador (como el de OpenAI o HuggingFace).
* Un token no siempre es igual a una palabra: A veces una sola palabra genera 2 o más tokens. Palabras muy comunes generan menos tokens.
* Es útil cuando trabajas con modelos LLM porque ellos procesan tokens, no caracteres.

¿Cuál elegir?

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/03/tabla_cursoRAG_2.png" width="800" /></center>

Más información en:
- https://python.langchain.com/api_reference/text_splitters/index.html
- https://python.langchain.com/docs/concepts/text_splitters/

<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License